### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Set your Azure Subscription](#1)
- [2️⃣ Create the Azure Resource Group](#2)
- [3️⃣ Add the aks-preview extension to the Azure CLI](#3)
- [4️⃣ Registration of preview features](#4)
- [5️⃣ Create and connect to an AKS Automatic Cluster](#5)
- [6️⃣ Verify the connection](#6)
- [7️⃣ Deploy an application](#7)
- [8️⃣ Test the application](#8)
- [🗑️ Clean up resources](#clean)

### Prerequisites
- [Python 3.8 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) and matplotlib installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively) and select the Azure Subscription that you want to use in the lab

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Provide either the tenant and the subscription IDs, before running the lab.
- Adjust the location parameters according your preferences and on the [product availability by Azure region](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management).
- Ensure the Nodepool VM sizes are compatible with the selected location. If 'aks_node_vm_sizes' is unspecified, the deployment process will identify three available SKUs and attempt to use the first one that successfully deploys.

In [5]:
import os
import datetime
import json
import subprocess

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "eastus2"
cluster_admin_role_id = "4abbcc35-e782-43d8-92c5-2d3f1bd2253f" # https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles#azure-kubernetes-service-cluster-user-role
aks_resource_name = "aks-automatic"
aks_resource_namespace = "aks-store"
aks_node_vm_sizes = [ "Standard_DC4ds_v3" ]
aks_store_demo_yaml = "https://raw.githubusercontent.com/Azure-Samples/aks-store-demo/main/aks-store-ingress-quickstart.yaml"

<a id='1'></a>
### 1️⃣ Set your Azure Subscription
Use this step to specify which Azure tenant and subscription should be used for subsequent Azure CLI commands.

In [12]:
current_user_stdout = ! az account show --query user.name --output tsv
current_user = current_user_stdout.n

current_user_objectid_stdout = ! az ad signed-in-user show --query id -o tsv
current_user_objectid = current_user_objectid_stdout.n


<a id='2'></a>
### 2️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [7]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())


✅ Azure Resource Group  lab-aks-automatic  created ⌚  12:22:47.775213


<a id='3'></a>
### 3️⃣ Add the aks-preview extension to the Azure CLI.
Extensions are additional features that you can add to the Azure CLI to augment its capabilities, often providing early access to new or experimental features not yet available in the main release. The `--name aks-preview` specifies the name of the extension to add, which in this case is `aks-preview`, a collection of preview features for managing Azure Kubernetes Service (AKS) clusters. The `--allow-preview true` flag explicitly allows the installation of preview features, acknowledging that these features might not be fully tested or supported.

In [8]:
! az extension add --name aks-preview --allow-preview true
! az extension update --name aks-preview --allow-preview true

<a id='4'></a>
### 4️⃣ Registration of preview features.
This step targets the registration of various preview features for Azure Kubernetes Service (AKS) under the Microsoft.ContainerService namespace.

In [9]:
! az feature register --namespace Microsoft.ContainerService --name EnableAPIServerVnetIntegrationPreview
! az feature register --namespace Microsoft.ContainerService --name NRGLockdownPreview
! az feature register --namespace Microsoft.ContainerService --name SafeguardsPreview
! az feature register --namespace Microsoft.ContainerService --name NodeAutoProvisioningPreview
! az feature register --namespace Microsoft.ContainerService --name DisableSSHPreview
! az feature register --namespace Microsoft.ContainerService --name AutomaticSKUPreview

# Verify the registration status by using the az feature show command. It takes a few minutes for the status to show Registered.
! az feature show --namespace Microsoft.ContainerService --name AutomaticSKUPreview

# Once the feature 'AutomaticSKUPreview' is registered, invoking 'az provider register -n Microsoft.ContainerService' is required to get the change propagated.
! az provider register -n Microsoft.ContainerService

{

D:\a\_work\1\s\build_scripts\windows\artifacts\cli\Lib\site-packages\cryptography/hazmat/backends/openssl/backend.py:17: UserWarning: You are using cryptography on a 32-bit Python on a 64-bit Windows Operating System. Cryptography will be significantly faster if you switch to using a 64-bit Python.



  "id": "/subscriptions/9d4a14de-67d7-4029-a3b4-7a7e3e6581cf/providers/Microsoft.Features/providers/Microsoft.ContainerService/features/EnableAPIServerVnetIntegrationPreview",
  "name": "Microsoft.ContainerService/EnableAPIServerVnetIntegrationPreview",
  "properties": {
    "state": "Registered"
  },
  "type": "Microsoft.Features/providers/features"
}
{
  "id": "/subscriptions/9d4a14de-67d7-4029-a3b4-7a7e3e6581cf/providers/Microsoft.Features/providers/Microsoft.ContainerService/features/NRGLockdownPreview",
  "name": "Microsoft.ContainerService/NRGLockdownPreview",
  "properties": {
    "state": "Registered"
  },
  "type": "Microsoft.Features/providers/features"
}


{
  "id": "/subscriptions/9d4a14de-67d7-4029-a3b4-7a7e3e6581cf/providers/Microsoft.Features/providers/Microsoft.ContainerService/features/SafeguardsPreview",
  "name": "Microsoft.ContainerService/SafeguardsPreview",
  "properties": {
    "state": "Registered"
  },
  "type": "Microsoft.Features/providers/features"
}


{
  "id": "/subscriptions/9d4a14de-67d7-4029-a3b4-7a7e3e6581cf/providers/Microsoft.Features/providers/Microsoft.ContainerService/features/NodeAutoProvisioningPreview",
  "name": "Microsoft.ContainerService/NodeAutoProvisioningPreview",
  "properties": {
    "state": "Registered"
  },
  "type": "Microsoft.Features/providers/features"
}


{
  "id": "/subscriptions/9d4a14de-67d7-4029-a3b4-7a7e3e6581cf/providers/Microsoft.Features/providers/Microsoft.ContainerService/features/DisableSSHPreview",
  "name": "Microsoft.ContainerService/DisableSSHPreview",
  "properties": {
    "state": "Registered"
  },
  "type": "Microsoft.Features/providers/features"
}


{
  "id": "/subscriptions/9d4a14de-67d7-4029-a3b4-7a7e3e6581cf/providers/Microsoft.Features/providers/Microsoft.ContainerService/features/AutomaticSKUPreview",
  "name": "Microsoft.ContainerService/AutomaticSKUPreview",
  "properties": {
    "state": "Registered"
  },
  "type": "Microsoft.Features/providers/features"
}


{
  "id": "/subscriptions/9d4a14de-67d7-4029-a3b4-7a7e3e6581cf/providers/Microsoft.Features/providers/Microsoft.ContainerService/features/AutomaticSKUPreview",
  "name": "Microsoft.ContainerService/AutomaticSKUPreview",
  "properties": {
    "state": "Registered"
  },
  "type": "Microsoft.Features/providers/features"
}


<a id='5'></a>
### 5️⃣ Create and connect to an AKS Automatic Cluster.
This step targets the registration of various preview features for Azure Kubernetes Service (AKS) under the Microsoft.ContainerService namespace.

In [10]:
def run_command(command):
    try:
        output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        return True, output.decode("utf-8")
    except subprocess.CalledProcessError as e:
        return False, e.output.decode("utf-8")
    
# Check if aks_node_vm_sizes is set  
try:  
    aks_node_vm_sizes  
except NameError:  
    aks_node_vm_sizes = None  
  
if not aks_node_vm_sizes:  
    # Get the list of available VM sizes for AKS in the westeurope region  
    success, output = run_command("az vm list-sizes --location {resource_group_location} --output json")  
  
    if success:  
        available_vm_sizes = json.loads(output)  
        # Filter VM sizes with at least 2 CPUs and get the first 3 results  
        filtered_vm_sizes = [vm['name'] for vm in available_vm_sizes if vm['numberOfCores'] >= 2][:3]  
        aks_node_vm_sizes = filtered_vm_sizes  
        print("Filtered VM sizes:", aks_node_vm_sizes)  
    else:  
        print("Failed to get available VM sizes. Error:", output)  
        aks_node_vm_sizes = []  

# Loop through SKUs and attempt to create AKS cluster
for aks_node_vm_size in aks_node_vm_sizes:
    print(f"Trying node vm size: {aks_node_vm_size}")
    
    bicep_parameters = {
        "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
        "contentVersion": "1.0.0.0",
        "parameters": {
            "clusterName": { "value": aks_resource_name },
            "location": { "value": resource_group_location },
            "nodeVmSize": { "value": aks_node_vm_size }
        }
    }
    
    with open('params.json', 'w') as bicep_parameters_file:
        bicep_parameters_file.write(json.dumps(bicep_parameters))

    success, output = run_command(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file \"main.bicep\" --parameters \"params.json\"")
    if success:
        print("AKS cluster created successfully with node vm size:", aks_node_vm_size)
        break
    else:
        print("Failed to create AKS cluster with node vm size:", aks_node_vm_size)
        print("Error:", output)

Trying node vm size: Standard_DC4ds_v3
AKS cluster created successfully with node vm size: Standard_DC4ds_v3


<a id='6'></a>
### 6️⃣ Get AKS credentials and verify the connection.
You need kubelogin credential plugin installed - see https://github.com/Azure/kubelogin.

Authentication & Authorization applied for AKS Automatic is "Microsoft Entra ID authentication with Azure RBAC".
Check your user and the role you may want to assign, before running the below script.

In [13]:
def run_command(command):  
    try:  
        output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)  
        return True, output.decode("utf-8")  
    except subprocess.CalledProcessError as e:  
        return False, e.output.decode("utf-8")

# Extract the AKS ID from the az aks show command output, filtering out any warning messages (if any).
def extract_aks_id(output):  
    """  
    Extracts the AKS ID from the az aks show command output, filtering out any warning messages.  
    """  
    lines = output.strip().split("\n")  
    for line in lines:  
        if line.startswith("/subscriptions/"):  
            return line.strip()  
    return None  

# Assign the Azure Kubernetes Service Cluster Admin Role to the current user  
# Get the AKS cluster ID  
success, aks_id_output = run_command(f"az aks show --resource-group {resource_group_name} --name {aks_resource_name} --query id -o tsv")  
if success:
    aks_id = extract_aks_id(aks_id_output)  
    if aks_id:  
        print(f"AKS ID: {aks_id}")  

    # Assign the Azure Kubernetes Service Cluster Admin Role to the current user  
    ! az role assignment create --assignee {current_user_objectid} --role "{cluster_admin_role_id}" --scope {aks_id}
else:  
    print("Failed to get AKS cluster ID. Error:", aks_id_output)

# Now, use kubelogin to authenticate with AAD  
success, kube_login_output = run_command("kubelogin convert-kubeconfig -l azurecli")  
if success:  
    print("✅ kubelogin successful:", kube_login_output)  
else:  
    print("Failed to use kubelogin. Error:", kube_login_output)  

! az aks get-credentials --resource-group {resource_group_name} --name {aks_resource_name} --overwrite-existing

AKS ID: /subscriptions/9d4a14de-67d7-4029-a3b4-7a7e3e6581cf/resourcegroups/lab-aks-automatic/providers/Microsoft.ContainerService/managedClusters/aks-automatic
{
  "condition": null,
  "conditionVersion": null,
  "createdBy": null,
  "createdOn": "2024-08-09T11:48:12.355924+00:00",
  "delegatedManagedIdentityResourceId": null,
  "description": null,
  "id": "/subscriptions/9d4a14de-67d7-4029-a3b4-7a7e3e6581cf/resourcegroups/lab-aks-automatic/providers/Microsoft.ContainerService/managedClusters/aks-automatic/providers/Microsoft.Authorization/roleAssignments/99c15e6f-87c3-45cb-8e25-b3bbaac06330",
  "name": "99c15e6f-87c3-45cb-8e25-b3bbaac06330",
  "principalId": "2919724f-d440-483b-89ba-192cfb82adb4",
  "principalType": "User",
  "resourceGroup": "lab-aks-automatic",
  "roleDefinitionId": "/subscriptions/9d4a14de-67d7-4029-a3b4-7a7e3e6581cf/providers/Microsoft.Authorization/roleDefinitions/4abbcc35-e782-43d8-92c5-2d3f1bd2253f",
  "scope": "/subscriptions/9d4a14de-67d7-4029-a3b4-7a7e3e658

In [14]:
! kubectl get nodes

^C


<a id='7'></a>
### 7️⃣ Deploy an application.
We will deploy the [AKS Store demo](https://github.com/Azure-Samples/aks-store-demo) application. We will use a manifest file to create all the objects required to run the application in the cluster.
The manifest includes the following Kubernetes deployments and services:
- Store front: Web application for customers to view products and place orders.
- Product service: Shows product information.
- Order service: Places orders.
- Rabbit MQ: Message queue for an order queue.

In [16]:
! az role assignment list --assignee {current_user_objectid}

[]


In [ ]:
# Create a namespace
! kubectl create namespace {aks_resource_namespace}

# Deploy the application
! kubectl apply -n {aks_resource_namespace} -f {aks_store_demo_yaml}

<a id='8'></a>
### 8️⃣ Test the application.
When the application runs, a Kubernetes service exposes the application front end to the internet. This process can take a few minutes to complete.

In [ ]:
# Check status of the deployed pods
! kubectl get pods -n {aks_resource_namespace}

# Check for a public IP address for the store-front application. We will monitor progress until the public IP address is available.
# Once the **ADDRESS** output is populated, the public IP address is available.
! kubectl get ingress store-front -n {aks_resource_namespace} --watch

NAME          CLASS                                HOSTS   ADDRESS         PORTS   AGE
store-front   webapprouting.kubernetes.azure.com   *       52.254.93.172   80      7m18s
